In [1]:
# 코랩용이므로 코랩에서 시험하는 것이 아니면 삭제
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Input, Flatten

from tensorflow.keras.applications import ResNet50V2
import glob
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# 모델 학습
def create_model():
  IMAGE_SIZE = 224

  input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))

  # 전이학습 (Transfer Learning)
  base_model = ResNet50V2( input_tensor=input, include_top = False, weights='imagenet' )

  # Get Trained ResNet Feature Extraction Output Tensor
  bm_output = base_model.output

  # Customize Food Model
  x = GlobalAveragePooling2D()(bm_output) # Flatten

  # FCL
  x = Dense(128, activation='relu')(x)
  x = Dense(64, activation='relu')(x)

  # Output Layer
  output = Dense(40, activation='softmax')(x)

  model = Model(inputs=input, outputs=output)
  adam = Adam(0.0001)

  model.compile(
      optimizer=adam,
      loss='categorical_crossentropy',
      metrics=['acc']
  )

  return model

In [4]:
# load model with best weights
model_loaded = create_model()
model_loaded.load_weights("/content/model_18-0.89.hdf5")

94668760/94668760 [==============================] - 1s 0us/step


In [9]:
def find_recipe(searched):
  categories = ["감바스", "김치전","김치찌개", "달걀말이","닭갈비", "돈까스", "동그랑땡", "된장국", "떡볶이", "라따뚜이", "리조또", "라멘", "라면","마제소바", "막국수", "멘보샤", "미역국","불고기", "사케동", "샐러드", "수제비","스테이크", "쌀국수", "야끼소바", "어묵볶음", "오코노미야키","월남쌈", "잔치국수", "짜글이", "짜장밥", "추어탕", "치즈등갈비", "칠리새우", "카레", "타코야키","탕수육", "탕후루","파스타", "피자", "허니브레드"]
  IMAGE_SIZE = 224
  image = cv2.imread(searched)
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image_224_224 = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
  test_tensor = image_224_224[np.newaxis, ...] / 255.0
  result = model_loaded.predict(test_tensor)
  result_idx = np.argsort(result, axis=1)[0, ::-1][:5]
  for idx in result_idx:
    print(categories[idx], result[0, idx])

In [10]:
# 사진 넣기 (예시는 김치찌개 구글링 사진)
find_recipe("/content/testin2.jpg")

1/1 [==============================] - 4s 4s/step
추어탕 0.88339615
짜글이 0.09583667
김치찌개 0.019445792
수제비 0.00022913245
된장국 0.00021646086
